# Cephalometric image analysis

In [ ]:
import os
import cv2
import glob
import base64
import time
import requests
import json
import urllib
import numpy as np
import matplotlib.pyplot as plt

## Define call rules
Please modify the following code blocks based on the information you obtained from us.

In [ ]:
# Chohotech service request URL, sent with the API documentation.
base_url = "<service request URL>"

# Chohotech file service URL, sent with the API documentation.
file_server_url = "<service file server URL>"

# The authentication header must be passed in. Please keep the TOKEN confidential!!! If it is leaked, please contact us immediately to reset it. All tasks using this TOKEN will be charged to your account.
zh_token = "<your company's service Token, sent with the contact>" # All API calls must be authenticated with the token.

user_group = "APIClient" # User group, usually named APIClient.

# Your company's user_id, sent with the API documentation.
user_id = "<your company's user_id>"

# If you have received creds.json, it will be read directly below.
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
json_call = {
    "spec_version": "1.0-snapshot",
    "spec_name": "ceph-analysis",
    "spec_group": "ceph",
    "user_group": user_group,
    "user_id": user_id
}

### Define Callback URL

Callback will send a POST request to the specified URL after the workflow is completed. The request content is a JSON object with 4 fields:

1. workflow_id (str): the corresponding workflow_id
2. metadata (dict): the metadata you passed when starting the workflow
3. success (bool): whether the workflow succeeded (true for success)
4. reason (str or null): If success is true, this item will be null. Otherwise, it will be a string representing the reason for failure.

If you need a callback, please uncomment the following code block.

In [ ]:
# json_call['metadata'] = { # (optional) metadata can be added here, which will be attached to the callback information. Each item in the dictionary is limited to 128 characters
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ # (optional) callback URL can be added here
#     {"url": "https://www.baidu.com"} # multiple callback URLs can be added here, each in the format {"url": "xxx"}
# ]

## Define Input/Output Blocks

The input block is defined by `input_data` in the JSON file. For two-dimensional images, we support the following **input** file formats:

```
"jpg"
"jpeg"
"png"
```

For two-dimensional images, inputs can be passed in two ways:

1. Upload the file to our file service system first, then write the file pointer into the JSON call string and call the API.
2. Pass the base64-encoded binary data directly in the JSON call string.

We **strongly recommend** using the first method for the following reasons:

1. Base64 encoding will increase the data stream size and increase network latency.
2. To ensure API performance, Choho will reject overly large API requests. Therefore, the second method will fail to call for large files (HTTP CODE 413).

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # Must specify postfix, i.e., the file extension
                        headers={"X-ZH-TOKEN": zh_token}) # Get the signed upload URL
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # Returns a single string JSON "string", can also use json.loads(resp.text)

    resp = requests.put(upload_url, data)  # No auth header is needed for uploading to the cloud storage service

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

json_call["input_data"] = {
    "image": upload_file('ceph.jpg')
}

# Submit request

Submit the request using the `/run` POST method. 

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

print(run_id)

# Wait for the request to complete

Use the polling API to wait for the request to complete, the API URL is `/run/{run_id}`.

**It is strongly recommended to use the callback method.** Refer to [Define Callback URL](#Define-Callback-URL), the callback method will send a callback message to the given URL at the end of the workflow, regardless of whether the workflow is successful or not, so you do not need to poll the result.

In [ ]:
# Step 2: Polling the status
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 180: # Maximum wait time: 3 minutes
    time.sleep(0.3) # Polling interval
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break

if not result['completed']:
    if result['failed']:
        raise ValueError("API error, reason: " + str(result['reason_public']))
    raise TimeoutError("API timeout")

print("API execution time: {}s".format(time.time()-start_time))

# Get the request result

Use `/data/{run_id}` to get all output data of this workflow, use `/data/{run_id}/{key}` to get a specific item of data.

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

# Visualization

The following is the output result display of the lateral cephalometric analysis API. The code above this part is common for all APIs that take 2D images as input, while the following part is specific to the processing and display of lateral cephalometric analysis output.

For specific output rules and fields of the lateral cephalometric analysis API, please refer to: https://www.chohotech.com/docs/cloud-en/#/module/ceph-1

In [ ]:
def vis_teeth(img, pred):
    kps = pred['kps']
    all_pts = []
    all_names = []
    vis = img.copy()
    for name, kp in kps.items():
        x, y = int(kp[0]), int(kp[1])
        cv2.circle(vis, (x, y), radius=3, color=(255, 255, 255), thickness=-1)
        all_pts.append(kp)
        all_names.append(name)
    for kp in json.loads(pred['meta'])['lower_jaw_contour']:
        x, y = int(kp[0]), int(kp[1])
        cv2.circle(vis, (x, y), radius=3, color=(255, 0, 0), thickness=-1)

    target_names = [
        ['100800', '100040', '100802', '100803', '100041', '100805', '100300', '100807', '100808', '100054', '100810', '100053', '100812', '100042', '100814', '100055', '100043', '100045', '100818', '100050'],
        ['100049', '100821', '100046', '100044', '100824', '100051', '100826', '100827', '100047', '100829', '100052', '100831', '100048', '100833', '100834', '100056', '100836'],
        ['100019', '100018', '100839', '100840', '100020', '100022', '100021', '100844', '100845', '100303', '100027', '100017', '100026', '100850', '100851', '100016', '100025', '100015', '100855', '100856', '100857', '100858', '100859', '100860', '100861', '100862'],
        ['100302', '100864', '100865', '100866', '100867', '100021'],
        ['100008', '100870', '100013', '100872', '100873', '100874', '100008'],
        ['100301', '100876', '100877', '100878', '100879', '100880', '100010', '100882', '100883', '100884', '100885', '100886', '100887', '100009', '100889', '100011', '100891', '100012'],
        ['100893', '100894', '100003', '100896', '100897', '100898', '100899', '100003'],
        ['100901', '100902', '100903', '100904', '100905', '100906', '100907', '100908', '100909', '100910', '100911', '100912', '100913', '100914', '100005', '100016', '100917', '100918', '100919', '100920', '100902', '100922'],
        ['100004', '100924', '100925', '100926', '100004'],
        ['100904', '100927', '100920', '100929'],
        ['100930', '100007', '100932', '100933', '100934'],
        ['100103', '100936', '100104', '100938', '100939', '100940', '100941', '100942', '100103'],
        ['100220', '100221', '100222', '100223', '100224', '100225', '100226', '100227', '100228', '100229', '100230', '100231', '100232', '100233', '100234', '100235', '100220']
    ]

    for target_list in target_names:
        for name in target_list[:-1]:
            next_name = target_list[target_list.index(name) + 1]
            if name in all_names and next_name in all_names:
                idx1 = all_names.index(name)
                idx2 = all_names.index(next_name)
                x1, y1 = int(all_pts[idx1][0]), int(all_pts[idx1][1])
                x2, y2 = int(all_pts[idx2][0]), int(all_pts[idx2][1])
                cv2.line(vis, (x1, y1), (x2, y2), color=(0,255,0), thickness=1)
    return vis

In [ ]:
img = cv2.imread('../../data/ceph.jpg')

In [ ]:
vis = vis_teeth(img, result['result'])

In [ ]:
plt.figure(figsize=(20, 14), dpi=80)
plt.imshow(vis[:,:,::-1])
plt.show()